In [2]:
%load_ext autoreload
%autoreload 2

In [24]:
import torch
import os
import pandas as pd
import torch.nn as nn
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split, Dataset
 
from tqdm import tqdm
from utils.london.process_data import build_train_test_loaders, build_train_test_loaders_2
from utils.london.training import CustomMAELoss, CustomMAPELoss, test_model, get_flow_forecasting_metrics
from utils.london.link_loads import get_graph_attributes, df_to_graph, build_quarter_hour_data, add_missing_nodes, add_missing_nodes_2
from utils.london.models import STGCN
# import networkx as nx

In [4]:
folder_path = "data/london/"
# Get graph attributes, create dfs from csv, process Link column and ordered dfs by time
num_nodes, edge_index, node_mapping, dfs = get_graph_attributes(folder_path)

In [5]:
MAE = CustomMAELoss()
MAPE = CustomMAPELoss()

In [6]:
window_size = 4

In [7]:
# Build the Data loader
graph_data = []
# each df should have the same dimension and same nodes at the same columns
for filename, df in dfs.items():
    df = add_missing_nodes_2(df, node_mapping, num_nodes) # add zeros row for missing nodes
    df_qhrs = build_quarter_hour_data(df, filename, num_nodes) # retourne 24*4 df avec ses paramètres temporel et le flow
    graph_data.append(df_qhrs)
graphs = [[df_to_graph(df, edge_index) for df in graph_data[i]] for i in range(len(graph_data))]  # Un graphe par quart d'heure

train_loader, test_loader = build_train_test_loaders_2(graphs, window_size, train_split=0.8)

### Try to take NLC as nodes key attributes

In [ ]:
folder_path = "data/london/"
# Get graph attributes, create dfs from csv, process Link column and ordered dfs by time
num_nodes, edge_index, node_mapping, dfs = get_graph_attributes(folder_path)

In [ ]:
# Build the Data loader
graph_data = []
# each df should have the same dimension and same nodes at the same columns
for filename, df in dfs.items():
    df = add_missing_nodes_2(df, node_mapping, num_nodes) # add zeros row for missing nodes
    df_qhrs = build_quarter_hour_data(df, filename, num_nodes) # retourne 24*4 df avec ses paramètres temporel et le flow
    graph_data.append(df_qhrs)
graphs = [[df_to_graph(df, edge_index) for df in graph_data[i]] for i in range(len(graph_data))]  # Un graphe par quart d'heure

train_loader, test_loader = build_train_test_loaders_2(graphs, window_size, train_split=0.8)

### Test 1

In [7]:
class ImprovedSTGCN(nn.Module):
    def __init__(self, node_features, hidden_dim, temporal_features_dim=10):
        super(ImprovedSTGCN, self).__init__()
        self.gcn = GCNConv(node_features, hidden_dim)  # GCN
        self.fc1 = nn.Linear(hidden_dim + temporal_features_dim, 4)  # Prédiction finale
        self.fc2 = nn.Linear(4 + node_features, 1)
        # self.bn1 = nn.BatchNorm1d(hidden_dim)

    def forward(self, graph, temporal_features):
        x = self.gcn(graph.x, graph.edge_index)  # GCN
        x = F.relu(x) # (num_nodes, hideen_dim)
        
        # Use temporal features and x as inputs of fc and use the fc layer
        x = torch.cat((x, temporal_features), dim=-1)
        x = self.fc1(x) 
        x = F.relu(x)
        x = torch.cat((x, graph.x), dim=-1)
        x = self.fc2(x) 
        x = F.relu(x)
        return x # return the flow for each nodes (num_nodes, 1)

In [8]:
model = ImprovedSTGCN(node_features=window_size, hidden_dim=16)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = MAE

num_epochs = 40

# Boucle d'entraînement
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0
    for graph, temporal_features in train_loader:
        temporal_features = temporal_features.squeeze()
        optimizer.zero_grad()
        output = model(graph, temporal_features)
        target = graph.y    
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()        
        train_loss += loss.item()

    scheduler.step()
    
    print(f"Epoch {epoch+1}, Train loss: {train_loss / len(train_loader)}")

    # Évaluation sur le jeu de test
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Désactivation des gradients pour l'évaluation
        for graph, temporal_features in test_loader:
            temporal_features = temporal_features.squeeze()  # Squeeze si nécessaire
            output = model(graph, temporal_features)  # Assure-toi de passer aussi les temporal_features
            target = graph.y
            loss = criterion(output, target)
            test_loss += loss.item()

    print(f"Epoch {epoch+1}, Test loss: {test_loss / len(test_loader)}")

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch 1, Train loss: 48.95288218941953


  2%|▎         | 1/40 [00:11<07:47, 11.98s/it]

Epoch 1, Test loss: 47.144519521130455
Epoch 2, Train loss: 23.777809886203872


  5%|▌         | 2/40 [00:23<07:27, 11.78s/it]

...

Epoch 38, Test loss: 15.262246692464466
Epoch 39, Train loss: 14.21717159277863


 98%|█████████▊| 39/40 [08:02<00:12, 12.15s/it]

Epoch 39, Test loss: 14.976269902691008
Epoch 40, Train loss: 14.184178984562555


100%|██████████| 40/40 [08:13<00:00, 12.34s/it]

Epoch 40, Test loss: 14.836333154212861


In [9]:
test_model(model, test_loader)

En utilisant la dernière valeur du flow comme prédiction :
Test MAE: 14.836332321166992
MAPE for high targets: 0.04389444959261233
MAE for low targets: 5.5507153447658295


### Test 2

In [12]:
class ImprovedSTGCN(nn.Module):
    def __init__(self, node_features, hidden_dim, temporal_features_dim=10):
        super(ImprovedSTGCN, self).__init__()
        self.gcn1 = GCNConv(node_features, hidden_dim)  # GCN
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim + temporal_features_dim + node_features, hidden_dim)  # Prédiction finale
        self.fc2 = nn.Linear(hidden_dim + node_features, 1)
        # self.bn1 = nn.BatchNorm1d(hidden_dim)

    def forward(self, graph, temporal_features):
        x = self.gcn1(graph.x, graph.edge_index)  # GCN
        x = F.relu(x) # (num_nodes, hideen_dim)
        x = self.gcn2(x, graph.edge_index)  # GCN
        x = F.relu(x) # (num_nodes, hideen_dim)
        
        # Use temporal features and x as inputs of fc and use the fc layer
        x = torch.cat((x, temporal_features, graph.x), dim=-1)
        x = self.fc1(x) 
        x = F.relu(x)
        x = torch.cat((x, graph.x), dim=-1)
        x = self.fc2(x)
        x = F.relu(x)
        return x # return the flow for each nodes (num_nodes, 1)

In [13]:
model = ImprovedSTGCN(node_features=window_size, hidden_dim=32)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = MAE

num_epochs = 500

# Boucle d'entraînement
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0
    for graph, temporal_features in train_loader:
        temporal_features = temporal_features.squeeze()
        optimizer.zero_grad()
        output = model(graph, temporal_features)
        target = graph.y    
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()        
        train_loss += loss.item()

    scheduler.step()
    
    print(f"Epoch {epoch+1}, Train loss: {train_loss / len(train_loader)}")

    # Évaluation sur le jeu de test
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Désactivation des gradients pour l'évaluation
        for graph, temporal_features in test_loader:
            temporal_features = temporal_features.squeeze()  # Squeeze si nécessaire
            output = model(graph, temporal_features)  # Assure-toi de passer aussi les temporal_features
            target = graph.y
            loss = criterion(output, target)
            test_loss += loss.item()

    print(f"Epoch {epoch+1}, Test loss: {test_loss / len(test_loader)}")

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch 1, Train loss: 42.767816162904104


  0%|          | 1/500 [00:17<2:21:41, 17.04s/it]

Epoch 1, Test loss: 25.983634329031386
Epoch 2, Train loss: 28.558133171863027


  0%|          | 2/500 [00:33<2:17:38, 16.58s/it]

...

Epoch 498, Test loss: 12.757928732841734
Epoch 499, Train loss: 12.431549349427224


100%|█████████▉| 499/500 [1:01:38<00:07,  7.10s/it]

Epoch 499, Test loss: 12.757928732841734
Epoch 500, Train loss: 12.431549349427224


100%|██████████| 500/500 [1:01:46<00:00,  7.41s/it]

Epoch 500, Test loss: 12.757928732841734


In [14]:
test_model(model, test_loader)

En utilisant la dernière valeur du flow comme prédiction :
Test MAE: 12.757927894592285
MAPE for high targets: 0.038881105119510304
MAE for low targets: 4.805690102633976


### Test 3

In [11]:
class DeepSTGCN(nn.Module):
    def __init__(self, node_features, hidden_dim, temporal_features_dim=10):
        super(DeepSTGCN, self).__init__()
        self.gcn1 = GCNConv(node_features, hidden_dim)  # GCN
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)
        self.gcn3 = GCNConv(hidden_dim, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim, 8)  # Prédiction finale
        self.fc2 = nn.Linear(8 + temporal_features_dim + node_features, 16)
        self.fc3 = nn.Linear(16 + node_features, 8)
        self.fc4 = nn.Linear(8 + node_features, 1)
        # self.bn1 = nn.BatchNorm1d(hidden_dim)

    def forward(self, graph, temporal_features):
        x = self.gcn1(graph.x, graph.edge_index)  # GCN
        x = F.tanh(x) # (num_nodes, hideen_dim)
        x = self.gcn2(x, graph.edge_index)  # GCN
        x = F.tanh(x) # (num_nodes, hideen_dim)
        x = self.gcn3(x, graph.edge_index)  # GCN
        x = F.tanh(x) # (num_nodes, hideen_dim)
        x = self.fc1(x)
        x = F.tanh(x)     
        x = torch.cat((x, temporal_features, graph.x), dim=-1)
        x = self.fc2(x)
        x = F.tanh(x)
        x = torch.cat((x, graph.x), dim=-1)
        x = self.fc3(x)
        x = F.tanh(x)
        x = torch.cat((x, graph.x), dim=-1)
        x = self.fc4(x)
        x = F.relu(x)
        return x # return the flow for each nodes (num_nodes, 1)

In [12]:
model = DeepSTGCN(node_features=window_size, hidden_dim=32)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = MAE

num_epochs = 500

# Boucle d'entraînement
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0
    for graph, temporal_features in train_loader:
        temporal_features = temporal_features.squeeze()
        optimizer.zero_grad()
        output = model(graph, temporal_features)
        target = graph.y    
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()        
        train_loss += loss.item()

    scheduler.step()
    
    print(f"Epoch {epoch+1}, Train loss: {train_loss / len(train_loader)}")

    # Évaluation sur le jeu de test
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Désactivation des gradients pour l'évaluation
        for graph, temporal_features in test_loader:
            temporal_features = temporal_features.squeeze()  # Squeeze si nécessaire
            output = model(graph, temporal_features)  # Assure-toi de passer aussi les temporal_features
            target = graph.y
            loss = criterion(output, target)
            test_loss += loss.item()

    print(f"Epoch {epoch+1}, Test loss: {test_loss / len(test_loader)}")

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch 1, Train loss: 42.802639022138386


  0%|          | 1/500 [00:23<3:15:14, 23.48s/it]

Epoch 1, Test loss: 22.921436099779037
Epoch 2, Train loss: 28.965290353298187


  0%|          | 2/500 [00:46<3:14:25, 23.42s/it]

...

Epoch 153, Test loss: 13.518963012430403
Epoch 154, Train loss: 13.331790215969086


 31%|███       | 154/500 [59:15<2:19:01, 24.11s/it]

Epoch 154, Test loss: 13.51892079815032


 31%|███       | 154/500 [59:32<2:13:47, 23.20s/it]


KeyboardInterrupt: 

In [13]:
test_model(model, test_loader)

Test MAE: 13.51826000213623
MAPE for high targets: 0.04106026636214838
MAE for low targets: 5.213425626357396


In [14]:
test_model(model, train_loader)

Test MAE: 13.328251838684082
MAPE for high targets: nan
MAE for low targets: 5.288397481242816


### Test 4

Modifs par rapport au meilleur modèle (Test 2): suffle les batchs

In [25]:
window_size = 4

In [26]:
# Build the Data loader
graph_data = []
# each df should have the same dimension and same nodes at the same columns
for filename, df in dfs.items():
    df = add_missing_nodes_2(df, node_mapping, num_nodes) # add zeros row for missing nodes
    df_qhrs = build_quarter_hour_data(df, filename, num_nodes) # retourne 24*4 df avec ses paramètres temporel et le flow
    graph_data.append(df_qhrs)
graphs = [[df_to_graph(df, edge_index) for df in graph_data[i]] for i in range(len(graph_data))]  # Un graphe par quart d'heure

train_loader, test_loader = build_train_test_loaders_2(graphs, window_size, train_split=0.8)

In [27]:
class ImprovedSTGCN(nn.Module):
    def __init__(self, node_features, hidden_dim, temporal_features_dim=10):
        super(ImprovedSTGCN, self).__init__()
        self.gcn1 = GCNConv(node_features, hidden_dim)  # GCN
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim + temporal_features_dim + node_features, hidden_dim)  # Prédiction finale
        self.fc2 = nn.Linear(hidden_dim + node_features, 1)
        # self.bn1 = nn.BatchNorm1d(hidden_dim)

    def forward(self, graph, temporal_features):
        x = self.gcn1(graph.x, graph.edge_index)  # GCN
        x = F.relu(x) # (num_nodes, hideen_dim)
        x = self.gcn2(x, graph.edge_index)  # GCN
        x = F.relu(x) # (num_nodes, hideen_dim)
        
        # Use temporal features and x as inputs of fc and use the fc layer
        x = torch.cat((x, temporal_features, graph.x), dim=-1)
        x = self.fc1(x) 
        x = F.relu(x)
        x = torch.cat((x, graph.x), dim=-1)
        x = self.fc2(x)
        x = F.relu(x)
        return x # return the flow for each nodes (num_nodes, 1)

In [ ]:
model = ImprovedSTGCN(node_features=window_size, hidden_dim=32)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = MAE

num_epochs = 500

# Boucle d'entraînement
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0
    for graph, temporal_features in train_loader:
        temporal_features = temporal_features.squeeze()
        optimizer.zero_grad()
        output = model(graph, temporal_features)
        target = graph.y    
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()        
        train_loss += loss.item()

    scheduler.step()
    
    print(f"Epoch {epoch+1}, Train loss: {train_loss / len(train_loader)}")

    # Évaluation sur le jeu de test
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Désactivation des gradients pour l'évaluation
        for graph, temporal_features in test_loader:
            temporal_features = temporal_features.squeeze()  # Squeeze si nécessaire
            output = model(graph, temporal_features)  # Assure-toi de passer aussi les temporal_features
            target = graph.y
            loss = criterion(output, target)
            test_loss += loss.item()

    print(f"Epoch {epoch+1}, Test loss: {test_loss / len(test_loader)}")

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch 1, Train loss: 27.358763943711917


  0%|          | 1/500 [00:17<2:26:26, 17.61s/it]

Epoch 1, Test loss: 20.31592579682668
Epoch 2, Train loss: 18.277280562387574


  0%|          | 2/500 [00:35<2:26:59, 17.71s/it]

...

Epoch 15, Test loss: 20.05636560680374
Epoch 16, Train loss: 14.002680289944013


  3%|▎         | 16/500 [04:36<2:17:59, 17.11s/it]

Epoch 16, Test loss: 14.541713777515623
Epoch 17, Train loss: 13.981220935649342


  3%|▎         | 17/500 [04:53<2:17:58, 17.14s/it]

Epoch 17, Test loss: 14.418298507020587


In [ ]:
test_model(model, test_loader)